In [10]:
import sys
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import skimage as ski
import PIL as PIL
import time

from skimage import measure
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.tri as mtri
from PIL import Image

np.set_printoptions(threshold = 1000)

Implement a function surface_transform, which takes two input arguments 

1) a triangulated surface, 
represented by a list of vertices and a list of triangles, and
2)  a vector of rigid transformation 
parameters, representing a rigid spatial transformaton

This function should return  triangulated 
surface that is transformed by the specified spatial transformation. In the function, comments shoud 
be included to explain the definition of the adopted coordinate system.

In [11]:
# Load the segmentation file “label_train00.npy” file. 

label_train = np.load(r"label_train00.npy")

# This loads a 32x 128 x 128 3d dataset. 

# The npy file stores data in binary format, so I have set the marching cubes "threshold" value to 0.5 - this will 
# contour all voxels represented by a value of 1

# Use sckit-image.measure.marching_cubes algorithm to compute vertex coordinates in mm and triangles for
# representing the segmentation boundary.

# Voxel dimensions are 2.0 mm x 0.5 mm x 0.5 mm 
# I have chosen to use a cartesian co-ordinate system, with the image positioned relative to the origin at
# (0, 0, 0) in the left bottom corner. Axis dimensions are displayed in mm. 

verts, faces, normals, values = measure.marching_cubes(label_train, 0.5, spacing = (2.0, 0.5, 0.5), step_size = 1)
print("Vertex co-ordinates (mm) are:  \n" + str(verts) + "\n")
print("Triangle indices are:  \n" + str(faces)+ "\n")

Vertex co-ordinates (mm) are:  
[[15.   20.   29.5 ]
 [16.   20.   29.25]
 [16.   19.75 29.5 ]
 ...
 [49.   25.   31.  ]
 [49.   25.   31.5 ]
 [49.   25.   32.  ]]

Triangle indices are:  
[[   2    1    0]
 [   4    3    0]
 [   0    3    2]
 ...
 [6493 6624 6495]
 [6495 6624 6625]
 [6495 6625 6470]]



In [12]:
# Display resulting triangular mesh using Matplotlib. 

vertices = np.array(verts)
indices = np.array(faces)

z = vertices[:, 0]
y = vertices[:, 1]
x = vertices[:, 2]


#fig0 = plt.figure()
#ax0 = fig0.add_subplot(111, projection='3d')

#ax0.set_zlim(np.min(z), np.max(z))
#ax0.set_ylim(np.min(y), np.max(y)) 
#ax0.invert_yaxis()     # y-axis inverted so that image is relative to the origin at (0, 0, 0)
#ax0.set_xlim(np.min(x), np.max(x))

#ax0.view_init(vertical_axis = "y")

#ax0.plot_trisurf(x, y, z, triangles = indices, color = "lavenderblush") 
#ax0.set_zlabel("Z (mm)")
#ax0.set_ylabel("Y (mm)")
#ax0.set_xlabel("X (mm)")
#plt.ioff()
#plt.show()

Implement a function surface_transform, which takes two input arguments 1) a triangulated surface, represented
by a list of vertices and a list of triangles, and 2) a vector of rigid transformation parameters.

In [13]:
# The "surface_transform" function firstly creates a "ones" matrix, which appends the image 
# co-ordinate matrix with a column of ones to transform the image co-ordinates in 3D relative to 
# the origin.

# The function then multiplies the "ones" matrix by a 4 x 4 transformation matrix incorporaring 
# rotation, R, and translation, t, i.e. new point P_new = R @ P0 + t

# Finally, the function removes the "ones" column to return the  matrix co-ordinates of the 
# transformed image.

def surface_transform(image, transform):
    ones = np.reshape(np.ones(len(image)),(len(image), 1))   
    image_ones = np.transpose(np.append(image, ones, axis = 1))
    image_transform = transform @ image_ones
    transform_transpose = np.transpose(image_transform)
    new_image = np.delete(transform_transpose, 3, axis = 1)
    return(new_image)

# Copy vertex coordinates using the new function, transforming the original surface by the identity matrix. 
# I0 will represent the co-ordinates of the copied surface.

T_I = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
               [0, 0, 0, 1]])

I = surface_transform(vertices, T_I)
I_triangles = surface_transform(indices, T_I)

z_I = I[:, 0]
y_I = I[:, 1]
x_I = I[:, 2]

#fig_I = plt.figure()
#ax_I = fig_I.add_subplot(111, projection='3d')

#ax_I.set_zlim(np.min(z_I), np.max(z_I))
#ax_I.set_ylim(np.min(y_I), np.max(y_I))
#ax_I.invert_yaxis()  
#ax_I.set_xlim(np.min(x_I), np.max(x_I))

#ax_I.plot_trisurf(x_I, y_I, z_I, triangles = indices, color = "lavenderblush") 

#ax_I.set_xlabel("X (mm)")
#ax_I.set_ylabel("Y (mm)")
#ax_I.set_zlabel("Z (mm)")
#ax_I.view_init(vertical_axis = "y")

#plt.savefig("original_surface.png", format = "png")  # Save plot as .png file
#plt.ioff()
#plt.show()

In [14]:
# Transform the images to real-world coordinates by applying a translation z_0 = (depth - 1)/2, 
# y_0 = (length - 1)/2 and x_0 = (width - 1)/2

z_0 = ((np.max(z_I) - np.min(z_I))-1)/2
y_0 = ((np.max(y_I) - np.min(y_I))-1)/2
x_0 = ((np.max(x_I) - np.min(x_I))-1)/2

T_0 = np.array([[1, 0, 0, -z_0],
              [0, 1, 0, -y_0],
              [0, 0, 1, -x_0],
               [0, 0, 0, 1]])

I0 = surface_transform(vertices, T_0)
   
z_I0 = I0[:, 0]
y_I0 = I0[:, 1]
x_I0 = I0[:, 2]

#fig_I0 = plt.figure()
#ax_I0 = fig_I0.add_subplot(111, projection='3d')

#ax_I0.set_zlim(np.min(z_I0), np.max(z_I0))
#ax_I0.set_ylim(np.min(y_I0), np.max(y_I0))
#ax_I0.invert_yaxis()  
#ax_I0.set_xlim(np.min(x_I0), np.max(x_I0))
#plt.ioff()
#ax_I0.set_zlim(np.minimum(np.min(z_I0),np.min(z_I)), np.maximum(np.max(z_I0),np.max(z_I)))
#ax_I0.set_ylim(np.minimum(np.min(y_I0),np.min(y_I)), np.maximum(np.max(y_I0),np.max(y_I)))
#ax_I0.invert_yaxis()  
#ax_I0.set_xlim(np.minimum(np.min(x_I0),np.min(x_I)), np.maximum(np.max(x_I0),np.max(x_I)))
#ax_I0.view_init(vertical_axis = "y")

#ax_I0.plot_trisurf(x_I, y_I, z_I, triangles = indices, color = "lavenderblush") 
#ax_I0.plot_trisurf(x_I0, y_I0, z_I0, triangles = indices, color = "lavenderblush") 

#ax_I0.set_zlabel("Z (mm)")
#ax_I0.set_xlabel("X (mm)")
#ax_I0.set_ylabel("Y (mm)")


#plt.savefig("world_coordinates.png", format = "png")  # Save plot as .png file

#plt.show()

In [15]:
# Surface transform 1. 
# Combine rotation of 60 degrees around the z-axis, 
                    # 90 degrees around the y- axis and 
                    # -45 degrees around the x-axis 
# with a translation of 50 mm in the z-direction, 
                      # 25 mm in the y-direction and 
                      #-125 mm in the x-direction

# Trigonometric functions in numpy require angles given in radians

theta_z = np.pi/3
theta_y = np.pi/2
theta_x = -(np.pi/4)

# Rotation around the z-axis
Rot_Z1 = np.array([[1, 0, 0, 0],
                  [0, np.cos(theta_z), -np.sin(theta_z), 0],
                  [0, np.sin(theta_z), np.cos(theta_z), 0],
                  [0, 0, 0, 1]])

# Rotation around the y-axis
Rot_Y1 = np.array([[np.cos(theta_y), 0, np.sin(theta_y), 0],
                  [0, 1 , 0, 0],
                  [-np.sin(theta_y), 0, np.cos(theta_y), 0],
                  [0, 0, 0, 1]])
# Rotation around the x-axis
Rot_X1 = np.array([[np.cos(theta_x), -np.sin(theta_x), 0, 0],
                  [np.sin(theta_x), np.cos(theta_x) , 0, 0],
                  [0, 0 , 1, 0],
                  [0, 0, 0, 1]])


Translate_1 = np.transpose(np.array([[1, 0, 0, 50],
                        [0, 1, 0, 25],
                        [0, 0, 1, -125],
                        [0, 0, 0, 1]]))

transform_1 = Rot_Z1 @ Rot_Y1 @ Rot_X1 @ Translate_1

I1 = surface_transform(I0, transform_1)

z_I1 = I1[:, 0]
y_I1 = I1[:, 1]
x_I1 = I1[:, 2]

#fig_I1 = plt.figure()
#ax_I1 = fig_I1.add_subplot(111, projection='3d')
#ax_I1.view_init(vertical_axis = "y")
#plt.ioff()
#ax_I1.set_zlim(np.minimum(np.min(z_I0),np.min(z_I1)), np.maximum(np.max(z_I0),np.max(z_I1)))
#ax_I1.set_ylim(np.minimum(np.min(y_I0),np.min(y_I1)), np.maximum(np.max(y_I0),np.max(y_I1)))
#ax_I1.invert_yaxis()  
#ax_I1.set_xlim(np.minimum(np.min(x_I0),np.min(x_I1)), np.maximum(np.max(x_I0),np.max(x_I1)))

#ax_I1.set_xlabel("X (mm)")
#ax_I1.set_ylabel("Y (mm)")
#ax_I1.set_zlabel("Z (mm)")

# Plot transformed surface, I1 (green) and original surface, I0 (pink):
#ax_I1.plot_trisurf(x_I1, y_I1, z_I1, triangles = indices, color = "honeydew")
#ax_I1.plot_trisurf(x_I0, y_I0, z_I0, triangles = indices, color = "lavenderblush") 
#plt.savefig("transformed_surface_1.png", format = "png")
#plt.show()



In [16]:
# Surface transform 2. 
# Combine rotation of 30 degrees around the z-axis, 
                    # 30 degrees around the y- axis and 
                    # 30 degrees around the x-axis 
# with a translation of 10 mm in the z-direction, 
                      # 10 mm in the y-direction and 
                      # 10 mm in the x-direction

# Trigonometric functions in numpy require angles given in radians

theta_z = np.pi/6
theta_y = np.pi/6
theta_x = np.pi/6

# Rotation around the z-axis
Rot_Z2 = np.array([[1, 0, 0, 0],
                  [0, np.cos(theta_z), -np.sin(theta_z), 0],
                  [0, np.sin(theta_z), np.cos(theta_z), 0],
                  [0, 0, 0, 1]])

# Rotation around the y-axis
Rot_Y2 = np.array([[np.cos(theta_y), 0, np.sin(theta_y), 0],
                  [0, 1 , 0, 0],
                  [-np.sin(theta_y), 0, np.cos(theta_y), 0],
                  [0, 0, 0, 1]])

# Rotation around the x-axis
Rot_X2 = np.array([[np.cos(theta_x), -np.sin(theta_x), 0, 0],
                  [np.sin(theta_x), np.cos(theta_x) , 0, 0],
                  [0, 0 , 1, 0],
                  [0, 0, 0, 1]])


Translate_2 = np.transpose(np.array([[1, 0, 0, 10],
                        [0, 1, 0, 10],
                        [0, 0, 1, 10],
                        [0, 0, 0, 1]]))

transform_2 = Rot_Z2 @ Rot_Y2 @ Rot_X2 @ Translate_2

I2 = surface_transform(I0, transform_2)

z_I2 = I2[:, 0]
y_I2 = I2[:, 1]
x_I2 = I2[:, 2]

#fig_I2 = plt.figure()
#ax_I2 = fig_I2.add_subplot(111, projection='3d')
#ax_I2.view_init(elev = 20, azim = -20, vertical_axis = "y")
#plt.ioff()
#ax_I2.set_zlim(np.minimum(np.min(z_I0),np.min(z_I2)), np.maximum(np.max(z_I0),np.max(z_I2)))
#ax_I2.set_ylim(np.minimum(np.min(y_I0),np.min(y_I2)), np.maximum(np.max(y_I0),np.max(y_I2)))
#ax_I2.invert_yaxis()  
#ax_I2.set_xlim(np.minimum(np.min(x_I0),np.min(x_I2)), np.maximum(np.max(x_I0),np.max(x_I2)))

#ax_I2.set_xlabel("X (mm)")
#ax_I2.set_ylabel("Y (mm)")
#ax_I2.set_zlabel("Z (mm)")

# Plot transformed surface, I2 (blue) and original surface, I0 (pink):
#ax_I2.plot_trisurf(x_I2, y_I2, z_I2, triangles = indices, color = "aliceblue")
#ax_I2.plot_trisurf(x_I0, y_I0, z_I0, triangles = indices, color = "lavenderblush") 
#plt.savefig("transformed_surface_2.png", format = "png")
#plt.show()

In [17]:
# Surface transform 3. 
# Combine rotation of 180 degrees around the z-axis, 
                    # 22.5 degrees around the y- axis and 
                    # 30 degrees around the x-axis 
# with a translation of 25 mm in the z-direction, 
                      # 15 mm in the y-direction and 
                      # -50 mm in the x-direction

# Trigonometric functions in numpy require angles given in radians

theta_z = np.pi
theta_y = -np.pi/8
theta_x = np.pi/6

# Rotation around the z-axis
Rot_Z3 = np.array([[1, 0, 0, 0],
                  [0, np.cos(theta_z), -np.sin(theta_z), 0],
                  [0, np.sin(theta_z), np.cos(theta_z), 0],
                  [0, 0, 0, 1]])

# Rotation around the y-axis
Rot_Y3 = np.array([[np.cos(theta_y), 0, np.sin(theta_y), 0],
                  [0, 1 , 0, 0],
                  [-np.sin(theta_y), 0, np.cos(theta_y), 0],
                  [0, 0, 0, 1]])

# Rotation around the x-axis
Rot_X3 = np.array([[np.cos(theta_x), -np.sin(theta_x), 0, 0],
                  [np.sin(theta_x), np.cos(theta_x) , 0, 0],
                  [0, 0 , 1, 0],
                  [0, 0, 0, 1]])


Translate_3 = np.transpose(np.array([[1, 0, 0, 25],
                        [0, 1, 0, 15],
                        [0, 0, 1, -50],
                        [0, 0, 0, 1]]))

transform_3 = Rot_X3 @ Rot_Y3 @ Rot_Z3 @ Translate_3

I3 = surface_transform(I0, transform_3)

z_I3 = I3[:, 0]
y_I3 = I3[:, 1]
x_I3 = I3[:, 2]

#fig_I3 = plt.figure()
#ax_I3 = fig_I3.add_subplot(111, projection='3d')
#ax_I3.view_init(elev = 20, azim = -20, vertical_axis = "y")
#plt.ioff()
#ax_I3.set_zlim(np.minimum(np.min(z_I0),np.min(z_I3)), np.maximum(np.max(z_I0),np.max(z_I3)))
#ax_I3.set_ylim(np.minimum(np.min(y_I0),np.min(y_I3)), np.maximum(np.max(y_I0),np.max(y_I3)))
#ax_I3.invert_yaxis()  
#ax_I3.set_xlim(np.minimum(np.min(x_I0),np.min(x_I3)), np.maximum(np.max(x_I0),np.max(x_I3)))

#ax_I3.set_xlabel("X (mm)")
#ax_I3.set_ylabel("Y (mm)")
#ax_I3.set_zlabel("Z (mm)")

# Plot transformed surface, I3 (yellow) and original surface I0 (pink) :
#ax_I3.plot_trisurf(x_I3, y_I3, z_I3, triangles = indices, color = "cornsilk")
#ax_I3.plot_trisurf(x_I0, y_I0, z_I0, triangles = indices, color = "lavenderblush") 
#plt.savefig("transformed_surface_3.png", format = "png")
#plt.show()